In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split    
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, BatchNormalization, Normalization

# Generación de datos ficticios
num_muestras = 1000  # Número de muestras en el conjunto de datos

# Características visuales: 128 características numéricas (x1)
caracteristicas_visuales = np.random.rand(num_muestras, 128)

# Características de texto: 256 características numéricas (x2)
caracteristicas_texto = np.random.rand(num_muestras, 256)

# Precio (Salida 1): Valor numérico (por ejemplo, precio de un producto) (y1)
precio = np.random.rand(num_muestras, 1)

# Categoría (Salida 2): 10 categorías posibles (codificadas en one-hot) y2
categorias = np.random.randint(0, 10, size=(num_muestras, 1))
categorias_one_hot = tf.keras.utils.to_categorical(categorias, num_classes=10)

x_temp1, x_test1, y_temp1, y_test1 = train_test_split(caracteristicas_visuales, precio, test_size=0.2, random_state=17)
x_train1, x_val1, y_train1, y_val1 = train_test_split(x_temp1, y_temp1, test_size=0.2, random_state=17)

x_temp2, x_test2, y_temp2, y_test2 = train_test_split(caracteristicas_texto, categorias_one_hot, test_size=0.2, random_state=17)
x_train2, x_val2, y_train2, y_val2 = train_test_split(x_temp2, y_temp2, test_size=0.2, random_state=17)

tf.keras.backend.clear_session()
tf.random.set_seed(17)

# Definición del modelo
input_layer_visual = tf.keras.layers.Input(shape=[128], name="Input_Visual") 
input_layer_texto = tf.keras.layers.Input(shape=[256], name="Input_Texto")  

hidden_visual = tf.keras.layers.Dense(64, activation="relu")(input_layer_visual)
hidden_texto = tf.keras.layers.Dense(128, activation="relu")(input_layer_texto)
concat = tf.keras.layers.concatenate([hidden_visual, hidden_texto])

# Salida categórica (usa `concat` en lugar de `hidden_texto`)
output_categorico = tf.keras.layers.Dense(10, activation="softmax", name="Saldia_Categorica")(concat)

# Salida de precio
output_precio = tf.keras.layers.Dense(1, name="Salida_Precio")(concat)

model = tf.keras.Model(inputs=[input_layer_visual, input_layer_texto], outputs=[output_categorico, output_precio])

# Ajuste del optimizador y la compilación
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)  # Prueba un valor más alto
model.compile(loss={"Saldia_Categorica": "sparse_categorical_crossentropy", "Salida_Precio": "mse"},
              optimizer=optimizer,
              metrics={"Saldia_Categorica": ["accuracy"], "Salida_Precio": ["RootMeanSquaredError"]})

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

# Entrenamiento: Asegúrate de pasar las entradas de manera correcta
history = model.fit(
    {"Input_Visual": x_train1, "Input_Texto": x_train2},  # Entradas explícitas
    {"Saldia_Categorica": y_train2, "Salida_Precio": y_train1},  # Salidas explícitas
    epochs=100,
    validation_data=(
        {"Input_Visual": x_val1, "Input_Texto": x_val2},  # Validación de entradas
        {"Saldia_Categorica": y_val2, "Salida_Precio": y_val1}  # Validación de salidas
    ),
    callbacks=[early_stopping_cb]
)


Epoch 1/100


ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 128, but received input with shape (32, 256)[0m

Arguments received by Functional.call():
  • inputs={'Input_Visual': 'tf.Tensor(shape=(32, 128), dtype=float32)', 'Input_Texto': 'tf.Tensor(shape=(32, 256), dtype=float32)'}
  • training=True
  • mask={'Input_Visual': 'None', 'Input_Texto': 'None'}